[28 批量归一化](https://www.bilibili.com/video/BV1X44y1r77r?spm_id_from=333.999.0.0)

- 之前提及的神经网络存在一个问题
    - 神经网络每层的梯度是不尽相同的，梯度变化一般是底层较大，顶层较小
    - 梯度变化的不一致会导致反向传播的时候，顶层变化速度较慢
    - 底层收敛速度虽然比较快，但是顶层每次发生变化，底层参数都需要重新训练
    - 很显然，我们希望有一种办法能够加快顶层的收敛速度

- <img src="picture\屏幕截图 2022-05-15 230011.png"></a>